## PostgreSQL + pgvector + Apache AGE の完全セットアップ成功

### to do
ベクトル検索は"HNSWインデックス"の方をデフォルトにした方が良さそう。

In [1]:
#@title セットアップ
# 1. システムの更新と必要なパッケージのインストール
!apt-get update
!apt-get install -y postgresql postgresql-contrib postgresql-server-dev-14
# → PostgreSQL 14 だから postgresql-server-dev-15 を dev-14 に変更
!apt-get install -y build-essential libreadline-dev zlib1g-dev flex bison git

# 2. PostgreSQLサーバーの起動
!service postgresql start

# 3. 必要なPythonライブラリのインストール
!pip install psycopg2-binary sqlalchemy asyncpg psycopg-pool "psycopg[binary,pool]"

print("✅ 基本システムのセットアップ完了")

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [9,104 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/restricted amd64 Packages [4,963 kB]
Get:13 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Package

In [2]:
#@title pgvector拡張機能のインストール
# pgvectorのソースからインストール
def install_pgvector():
    print("pgvector拡張機能をインストール中...")

    # pgvectorのクローンとビルド
    !git clone https://github.com/pgvector/pgvector.git /tmp/pgvector

    import os
    os.chdir('/tmp/pgvector')
    !make
    !make install

    print("✅ pgvector拡張機能のインストール完了")

install_pgvector()

pgvector拡張機能をインストール中...
Cloning into '/tmp/pgvector'...
remote: Enumerating objects: 11859, done.
remote: Counting objects: 100% (4919/4919), done.
remote: Compressing objects: 100% (312/312), done.
remote: Total 11859 (delta 4739), reused 4617 (delta 4607), pack-reused 6940 (from 2)
Receiving objects: 100% (11859/11859), 1.79 MiB | 9.61 MiB/s, done.
Resolving deltas: 100% (8902/8902), done.
gcc -Wall -Wmissing-prototypes -Wpointer-arith -Wdeclaration-after-statement -Werror=vla -Wendif-labels -Wmissing-format-attribute -Wimplicit-fallthrough=3 -Wcast-function-type -Wformat-security -fno-strict-aliasing -fwrapv -fexcess-precision=standard -Wno-format-truncation -Wno-stringop-truncation -g -g -O2 -flto=auto -ffat-lto-objects -flto=auto -ffat-lto-objects -fstack-protector-strong -Wformat -Werror=format-security -fno-omit-frame-pointer -march=native -ftree-vectorize -fassociative-math -fno-signed-zeros -fno-trapping-math -fPIC -I. -I./ -I/usr/include/postgresql/14/server -I/usr/include/po

In [3]:
#@title Apache AGE拡張機能のインストール
import os
import subprocess

def install_apache_age_pg14_v1_5_0_rc0():
    print("Apache AGE (PG14/v1.5.0-rc0) 拡張機能をインストール中...")

    # 前回の試行で作成されたディレクトリがあれば削除
    if os.path.exists('/tmp/age'):
        print("既存の /tmp/age ディレクトリをクリーンアップします...")
        subprocess.run(['rm', '-rf', '/tmp/age'])

    # Apache AGEリポジトリのクローン
    print("Apache AGE のリポジトリをクローンしています...")
    subprocess.run(['git', 'clone', 'https://github.com/apache/age.git', '/tmp/age'], check=True)

    os.chdir('/tmp/age')

    # 正しいタグ 'PG14/v1.5.0-rc0' に切り替え
    # これが修正点です
    tag_name = 'PG14/v1.5.0-rc0'
    print(f"リリース {tag_name} に切り替えています...")
    try:
        subprocess.run(['git', 'checkout', tag_name], check=True)
    except subprocess.CalledProcessError as e:
        print(f"git checkout中にエラーが発生しました。タグ '{tag_name}' が存在するか確認してください。")
        print(f"エラー詳細: {e}")
        return

    # PG_CONFIGを指定してPGXS経由でビルド
    print("ビルドを開始します...")
    try:
        subprocess.run(['make', 'PG_CONFIG=/usr/bin/pg_config'], check=True)
    except subprocess.CalledProcessError as e:
        print(f"ビルド中にエラーが発生しました: {e}")
        return

    # インストール
    print("インストールを実行します...")
    try:
        subprocess.run(['sudo', 'make', 'install', 'PG_CONFIG=/usr/bin/pg_config'], check=True)
    except subprocess.CalledProcessError as e:
        print(f"インストール中にエラーが発生しました: {e}")
        return

    print(f"✅ Apache AGE {tag_name} 拡張機能のインストール完了")

# 関数を実行
install_apache_age_pg14_v1_5_0_rc0()

Apache AGE (PG14/v1.5.0-rc0) 拡張機能をインストール中...
Apache AGE のリポジトリをクローンしています...
リリース PG14/v1.5.0-rc0 に切り替えています...
ビルドを開始します...
インストールを実行します...
✅ Apache AGE PG14/v1.5.0-rc0 拡張機能のインストール完了


In [4]:
#@title PostgreSQL設定の更新
def configure_postgresql():
    try:
        # 14系の設定ファイルパスに変更
        config_path = "/etc/postgresql/14/main/postgresql.conf"

        # shared_preload_librariesに 'age' を追加
        !sudo sed -i "s/#shared_preload_libraries = ''/shared_preload_libraries = 'age'/" {config_path}

        # PostgreSQLサービスの再起動
        !sudo service postgresql restart

        print("✅ PostgreSQL設定を更新しました")

    except Exception as e:
        print(f"設定更新エラー: {e}")

configure_postgresql()

 * Restarting PostgreSQL 14 database server
   ...done.
✅ PostgreSQL設定を更新しました


In [5]:
#@title データベースとユーザーの設定
import psycopg2
from sqlalchemy import create_engine, text
import asyncpg

# データベース接続の設定
DB_CONFIG = {
    'host': 'localhost',
    'port': 5432,
    'database': 'minirag_db',
    'user': 'postgres',
    'password': 'postgres'
}

# 基本的なデータベース設定
def setup_database():
    try:
        # postgresユーザーでの初期設定
        !sudo -u postgres psql -c "CREATE USER root WITH SUPERUSER"
        !sudo -u postgres psql -c "ALTER USER postgres PASSWORD 'postgres';"
        !sudo -u postgres psql -c "CREATE DATABASE minirag_db;"

        print("✅ データベースとユーザーの設定完了")

    except Exception as e:
        print(f"データベース設定エラー: {e}")

setup_database()

CREATE ROLE
ALTER ROLE
CREATE DATABASE
✅ データベースとユーザーの設定完了


In [6]:
#@title pgvectorとApache AGE拡張機能の有効化
def enable_extensions():
    try:
        conn = psycopg2.connect(**DB_CONFIG)
        cursor = conn.cursor()

        # AGE拡張機能をロード
        cursor.execute("LOAD 'age';")

        # 拡張機能を作成
        cursor.execute("CREATE EXTENSION IF NOT EXISTS vector;")
        cursor.execute("CREATE EXTENSION IF NOT EXISTS age;")

        # search_pathの設定[2]
        cursor.execute("SET search_path = ag_catalog, \"$user\", public;")

        # 拡張機能の確認
        cursor.execute("SELECT extname, extversion FROM pg_extension WHERE extname IN ('vector', 'age');")
        extensions = cursor.fetchall()

        print("インストール済み拡張機能:")
        for ext in extensions:
            print(f"  - {ext[0]} (バージョン: {ext[1]})")

        conn.commit()
        cursor.close()
        conn.close()

        print("✅ 拡張機能の有効化完了")

    except Exception as e:
        print(f"拡張機能有効化エラー: {e}")

enable_extensions()

インストール済み拡張機能:
  - vector (バージョン: 0.8.0)
  - age (バージョン: 1.5.0)
✅ 拡張機能の有効化完了


In [7]:
#@title Apache AGE用のグラフデータベース作成
def create_graph():
    try:
        conn = psycopg2.connect(**DB_CONFIG)
        cursor = conn.cursor()

        # search_pathの設定
        cursor.execute("SET search_path = ag_catalog, \"$user\", public;")

        # グラフの作成
        cursor.execute("SELECT create_graph('my_minirag_graph');")

        # 作成されたグラフの確認
        cursor.execute("SELECT name FROM ag_graph;")
        graphs = cursor.fetchall()
        print(f"作成済みグラフ: {[graph[0] for graph in graphs]}")

        conn.commit()
        cursor.close()
        conn.close()

        print("✅ グラフデータベースの作成完了")

    except Exception as e:
        print(f"グラフ作成エラー: {e}")

create_graph()

作成済みグラフ: ['my_minirag_graph']
✅ グラフデータベースの作成完了


In [8]:
#@title ベクトル検索とグラフ用のテーブル作成
def create_tables():
    try:
        conn = psycopg2.connect(**DB_CONFIG)
        cursor = conn.cursor()

        # ドキュメント用テーブル（ベクトル検索対応）
        cursor.execute("""
        CREATE TABLE IF NOT EXISTS documents (
            id SERIAL PRIMARY KEY,
            content TEXT NOT NULL,
            embedding vector(1536),  -- OpenAI embeddings用
            metadata JSONB,
            created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
        );
        """)

        # ベクトル検索用インデックス
        cursor.execute("""
        CREATE INDEX IF NOT EXISTS documents_embedding_idx
        ON documents USING ivfflat (embedding vector_cosine_ops)
        WITH (lists = 100);
        """)

        # エンティティ用テーブル（グラフ連携用）
        cursor.execute("""
        CREATE TABLE IF NOT EXISTS entities (
            id SERIAL PRIMARY KEY,
            name VARCHAR(255) NOT NULL,
            entity_type VARCHAR(100),
            properties JSONB,
            created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
        );
        """)

        # 関係性用テーブル
        cursor.execute("""
        CREATE TABLE IF NOT EXISTS relationships (
            id SERIAL PRIMARY KEY,
            source_entity_id INTEGER REFERENCES entities(id),
            target_entity_id INTEGER REFERENCES entities(id),
            relationship_type VARCHAR(100),
            weight FLOAT DEFAULT 1.0,
            properties JSONB,
            created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
        );
        """)

        conn.commit()
        cursor.close()
        conn.close()

        print("✅ テーブル構造の作成完了")

    except Exception as e:
        print(f"テーブル作成エラー: {e}")

create_tables()

✅ テーブル構造の作成完了


In [9]:
#@title サンプルデータの挿入とテスト
import json
import numpy as np

# サンプルデータの挿入とテスト
def insert_sample_data():
    try:
        conn = psycopg2.connect(**DB_CONFIG)
        cursor = conn.cursor()

        # 1. ベクトルデータの挿入テスト
        sample_embedding = np.random.rand(1536).tolist()  # ダミーベクトル

        cursor.execute("""
        INSERT INTO documents (content, embedding, metadata)
        VALUES (%s, %s, %s)
        """, (
            "これはMiniRAGのテストドキュメントです。",
            json.dumps(sample_embedding),
            json.dumps({"type": "test", "category": "sample"})
        ))

        # 2. エンティティデータの挿入
        cursor.execute("""
        INSERT INTO entities (name, entity_type, properties)
        VALUES (%s, %s, %s), (%s, %s, %s)
        """, (
            "MiniRAG", "システム", json.dumps({"description": "RAGシステム"}),
            "PostgreSQL", "データベース", json.dumps({"description": "リレーショナルデータベース"})
        ))

        # 3. 関係性データの挿入
        cursor.execute("""
        INSERT INTO relationships (source_entity_id, target_entity_id, relationship_type, weight)
        VALUES (1, 2, 'USES', 0.9)
        """)

        # 4. Apache AGEでのグラフノード作成
        cursor.execute("SET search_path = ag_catalog, \"$user\", public;")

        cursor.execute("""
        SELECT * FROM cypher('my_minirag_graph', $$
        CREATE (d:Document {
            id: 1,
            content: 'これはMiniRAGのテストドキュメントです',
            type: 'sample'
        })
        $$) AS (result agtype);
        """)

        cursor.execute("""
        SELECT * FROM cypher('my_minirag_graph', $$
        CREATE (s:System {
            id: 1,
            name: 'MiniRAG',
            type: 'RAG_System'
        })
        $$) AS (result agtype);
        """)

        # 5. グラフエッジの作成
        cursor.execute("""
        SELECT * FROM cypher('my_minirag_graph', $$
        MATCH (d:Document), (s:System)
        WHERE d.id = 1 AND s.id = 1
        CREATE (d)-[r:BELONGS_TO {confidence: 0.95}]->(s)
        $$) AS (result agtype);
        """)

        conn.commit()
        cursor.close()
        conn.close()

        print("✅ サンプルデータの挿入完了")

    except Exception as e:
        print(f"サンプルデータ挿入エラー: {e}")

insert_sample_data()

✅ サンプルデータの挿入完了


In [10]:
#@title 検索機能のテスト

# ベクトル検索: documents という名前のリレーショナルテーブルを検索しています。
##### SELECT ... FROM documents ...
# グラフクエリ: my_minirag_graph という名前のグラフデータベースを検索しています。
##### SELECT * FROM cypher('my_minirag_graph', $$ ... $$)

# ベクトル検索とグラフクエリのテスト
def test_search_functionality():
    try:
        conn = psycopg2.connect(**DB_CONFIG)
        cursor = conn.cursor()

        print("=== ベクトル検索テスト ===")
        # ベクトル類似性検索のテスト
        test_vector = np.random.rand(1536).tolist()

        #【修正点】
        # 'embedding %s' という不正な構文を、pgvectorのコサイン距離演算子 '<=>' に変更します。
        # これにより、'embedding' 列のベクトルと、パラメータとして渡される 'test_vector' との
        # コサイン距離が計算されます。
        cursor.execute("""
        SELECT id, content, embedding <=> %s AS distance
        FROM documents
        ORDER BY embedding <=> %s
        LIMIT 3;
        """, (json.dumps(test_vector), json.dumps(test_vector))) # 引数は2つ必要

        vector_results = cursor.fetchall()
        print(f"ベクトル検索結果: {len(vector_results)}件")
        # 挿入したサンプルデータが1件だけなので、結果も1件になります
        for result in vector_results:
            # result[2]がdistance
            print(f"  ID: {result[0]}, コンテンツ: '{result[1]}', 距離: {result[2]:.4f}")

        print("\n=== グラフクエリテスト ===")
        # Apache AGEでのグラフクエリ
        cursor.execute("SET search_path = ag_catalog, \"$user\", public;")

        cursor.execute("""
        SELECT * FROM cypher('my_minirag_graph', $$
        MATCH (d:Document)-[r:BELONGS_TO]->(s:System)
        RETURN d.content, r.confidence, s.name
        $$) AS (doc_content agtype, confidence agtype, system_name agtype);
        """)

        graph_results = cursor.fetchall()
        print(f"グラフクエリ結果: {len(graph_results)}件")
        for result in graph_results:
            print(f"  ドキュメント: {result[0]}, 信頼度: {result[1]}, システム: {result[2]}")

        cursor.close()
        conn.close()

        print("\n✅ 検索機能のテスト完了")

    except Exception as e:
        print(f"検索テストエラー: {e}")

test_search_functionality()

=== ベクトル検索テスト ===
ベクトル検索結果: 0件

=== グラフクエリテスト ===
グラフクエリ結果: 1件
  ドキュメント: "これはMiniRAGのテストドキュメントです", 信頼度: 0.95, システム: "MiniRAG"

✅ 検索機能のテスト完了


In [11]:
#@title データ確認用（挿入したデータがちゃんと入っているか）

def check_data():
    try:
        conn = psycopg2.connect(**DB_CONFIG)
        cursor = conn.cursor()

        print("--- 'documents' テーブルの確認 ---")
        cursor.execute("SELECT COUNT(*) FROM documents;")
        count = cursor.fetchone()[0]
        print(f"documents テーブルのレコード数: {count}")

        if count > 0:
            # ::textでベクトルを文字列にキャストし、先頭60文字だけ表示
            cursor.execute("SELECT id, content, left(embedding::text, 60) as embedding_preview, metadata FROM documents;")
            records = cursor.fetchall()
            print("テーブルの内容 (先頭レコード):")
            for rec in records:
                print(f"  ID: {rec[0]}, Content: '{rec[1]}', Embedding: '{rec[2]}...', Metadata: {rec[3]}")
        else:
            print("\n[原因] 'documents' テーブルは空です。これがベクトル検索で0件になる原因です。")


        print("\n--- 'my_minirag_graph' のノード確認 ---")
        cursor.execute("SET search_path = ag_catalog, \"$user\", public;")
        cursor.execute("""
            SELECT * FROM cypher('my_minirag_graph', $$
            MATCH (n)
            RETURN labels(n), n.content
            $$) AS (labels agtype, content agtype);
        """)
        graph_nodes = cursor.fetchall()
        print(f"my_minirag_graph のノード数: {len(graph_nodes)}")
        if len(graph_nodes) > 0:
            print("グラフの内容:")
            for node in graph_nodes:
                print(f"  Labels: {node[0]}, Content: {node[1]}")

        cursor.close()
        conn.close()

    except Exception as e:
        print(f"データ確認中にエラーが発生しました: {e}")

check_data()

--- 'documents' テーブルの確認 ---
documents テーブルのレコード数: 1
テーブルの内容 (先頭レコード):
  ID: 1, Content: 'これはMiniRAGのテストドキュメントです。', Embedding: '[0.29788288,0.57865953,0.70901406,0.83297515,0.19000451,0.24...', Metadata: {'type': 'test', 'category': 'sample'}

--- 'my_minirag_graph' のノード確認 ---
my_minirag_graph のノード数: 2
グラフの内容:
  Labels: ["Document"], Content: "これはMiniRAGのテストドキュメントです"
  Labels: ["System"], Content: None


In [12]:
# Systemノードを作成する部分では、Systemというラベルを持つノードに、id, name, type という3つのプロパティ（属性）を設定しています。
# ここには content というプロパティは含まれていません。
# Systemノードにはcontentプロパティが設定されていないため、データベースは「存在しない」という意味でNULLを返します。Pythonのpsycopg2ライブラリは、このNULLをNoneとして表示します。
# → つまり Labels: ["System"], Content: None は正常。

# -----------------------------------------------------------------------------

# === ベクトル検索テスト ===
# ベクトル検索結果: 0件  の理由

# 結論：バグではなく、近似最近傍探索(ANN)インデックスの特性によるものです。
# 原因：IVFFlatインデックスの仕組み
# 今回使用しているIVFFlatインデックスは、全データをスキャンするのではなく、高速化のためにデータをいくつかの「クラスタ（グループ）」に分割して管理します。
# インデックス作成時: データベース内の全ベクトルデータを元に、代表的なベクトル（クラスタの中心）を複数個（今回はlists = 100で100個）作成します。そして、各データがどのクラスタに最も近いかを記録します。
# 検索時:
# まず、検索したいベクトル（test_vector）が、どのクラスタに最も近いかを計算します。
# 次に、その近いいくつかのクラスタの中だけを検索して、似ているベクトルを探します。
# 問題点: データが1件しかない場合、この「クラスタリング」がうまく機能しません。100個のクラスタのうち、たった1つのデータがどこか1つにポツンと存在する状態です。検索用のtest_vector（ランダムに生成）が、そのデータが存在するクラスタの「近く」にあると判定されない限り、検索対象から漏れてしまい、結果が0件になってしまうのです。

In [13]:
#@title ベクトル検索機能の再テスト（インデックスを無視）

# 試しに 近似最近傍探索(ANN) を使わないで全件検索してみる
def test_search_functionality_seqscan():
    try:
        conn = psycopg2.connect(**DB_CONFIG)
        cursor = conn.cursor()

        print("=== ベクトル検索テスト（インデックスを無視） ===")
        # この設定で、PostgreSQLはインデックスを使わず全件検索します
        cursor.execute("SET LOCAL enable_indexscan = off;")

        test_vector = np.random.rand(1536).tolist()

        cursor.execute("""
        SELECT id, content, embedding <=> %s AS distance
        FROM documents
        ORDER BY embedding <=> %s
        LIMIT 3;
        """, (json.dumps(test_vector), json.dumps(test_vector)))

        vector_results = cursor.fetchall()
        print(f"ベクトル検索結果: {len(vector_results)}件") # 今度は1件ヒットするはず
        for result in vector_results:
            print(f"  ID: {result[0]}, コンテンツ: '{result[1]}', 距離: {result[2]:.4f}")

        # 他のテストは省略
        cursor.close()
        conn.close()

        print("\n✅ インデックスを無視した検索テスト完了")

    except Exception as e:
        print(f"検索テストエラー: {e}")

test_search_functionality_seqscan()

=== ベクトル検索テスト（インデックスを無視） ===
ベクトル検索結果: 1件
  ID: 1, コンテンツ: 'これはMiniRAGのテストドキュメントです。', 距離: 0.2437

✅ インデックスを無視した検索テスト完了


### HNSW (Hierarchical Navigable Small World) は、IVFFlatよりも構築に少し時間がかかりますが、検索性能が高く、特に今回のようにデータが少ない場合でも安定して結果を返してくれる非常に優れたアルゴリズム

試しに IVFFlat インデックス (documents_embedding_idx) を削除して HNSW でやってみる。

In [14]:
#@title HNSWインデックスへの切り替えとテスト

# 1.既存の IVFFlat インデックス (documents_embedding_idx) を削除します。
# 2.新しく HNSW インデックスを documents_embedding_idx という同じ名前で作成します。
# 3.HNSW インデックスが有効な状態で、もう一度検索テストを実行します。

def switch_to_hnsw_index():
    """
    既存のIVFFlatインデックスを削除し、HNSWインデックスを再作成します。
    """
    try:
        conn = psycopg2.connect(**DB_CONFIG)
        cursor = conn.cursor()

        print("--- HNSWインデックスへの切り替えを開始 ---")

        # 1. 既存のインデックスを削除
        print("1. 既存の'documents_embedding_idx'を削除しています...")
        # 'IF EXISTS' を付けることで、インデックスが存在しない場合にエラーになるのを防ぎます
        cursor.execute("DROP INDEX IF EXISTS documents_embedding_idx;")
        print("   -> 削除完了。")

        # 2. HNSWインデックスを作成
        print("2. 新しいHNSWインデックスを作成しています...")
        # HNSWインデックスは、m（接続数）とef_construction（構築時の探索範囲）というパラメータを持ちます。
        # これらはインデックスの精度と構築時間/メモリ使用量のトレードオフになります。
        # ここでは一般的なデフォルト値を使用します。
        cursor.execute("""
        CREATE INDEX documents_embedding_idx
        ON documents
        USING hnsw (embedding vector_cosine_ops)
        WITH (m = 16, ef_construction = 64);
        """)
        print("   -> HNSWインデックス作成完了。")

        conn.commit()
        cursor.close()
        conn.close()

        print("\n✅ HNSWインデックスへの切り替えが完了しました。")

    except Exception as e:
        print(f"HNSWインデックス切り替え中にエラーが発生しました: {e}")


# 1. HNSWインデックスに切り替える関数を実行します
switch_to_hnsw_index()

# 2. 再度、通常の検索テストを実行します
#    HNSWインデックスはデータが1件でも正しく機能するため、
#    以前のようにインデックスを無視する必要はありません。
print("\n--- HNSWインデックスを使用して検索テストを実行 ---")
# 以前定義した検索テスト関数を再利用します
test_search_functionality()

--- HNSWインデックスへの切り替えを開始 ---
1. 既存の'documents_embedding_idx'を削除しています...
   -> 削除完了。
2. 新しいHNSWインデックスを作成しています...
   -> HNSWインデックス作成完了。

✅ HNSWインデックスへの切り替えが完了しました。

--- HNSWインデックスを使用して検索テストを実行 ---
=== ベクトル検索テスト ===
ベクトル検索結果: 1件
  ID: 1, コンテンツ: 'これはMiniRAGのテストドキュメントです。', 距離: 0.2481

=== グラフクエリテスト ===
グラフクエリ結果: 1件
  ドキュメント: "これはMiniRAGのテストドキュメントです", 信頼度: 0.95, システム: "MiniRAG"

✅ 検索機能のテスト完了


In [15]:
# IVFFlatインデックス だとベクトル検索がゼロ件だったのが、
# HNSWインデックス だとベクトル検索で1件返るようになった。

In [16]:
#@title 非同期接続のテスト
import asyncio

# asyncpgを使用した非同期操作のテスト
async def test_async_operations():
    try:
        # 非同期接続
        conn = await asyncpg.connect(
            host=DB_CONFIG['host'],
            port=DB_CONFIG['port'],
            database=DB_CONFIG['database'],
            user=DB_CONFIG['user'],
            password=DB_CONFIG['password']
        )

        # 非同期でのデータ取得
        result = await conn.fetchval('SELECT version()')
        print(f"✅ 非同期接続成功: PostgreSQL {result.split()[1]}")

        # 非同期でのベクトル検索
        rows = await conn.fetch("""
        SELECT id, content, metadata
        FROM documents
        LIMIT 3
        """)

        print(f"✅ 非同期データ取得: {len(rows)}件のドキュメント")

        await conn.close()

    except Exception as e:
        print(f"非同期操作エラー: {e}")

# 非同期関数の実行
await test_async_operations()

✅ 非同期接続成功: PostgreSQL 14.18
✅ 非同期データ取得: 1件のドキュメント


In [17]:
#@title 最終確認とバックアップ
# from google.colab import drive
import pandas as pd

# Google Driveマウント
# drive.mount('/content/drive')

# 最終的な動作確認とバックアップ
def final_verification_and_backup():
    print("=== 最終確認 ===")

    conn = psycopg2.connect(**DB_CONFIG)
    cursor = conn.cursor()

    # 1. 拡張機能の確認
    cursor.execute("SELECT extname, extversion FROM pg_extension WHERE extname IN ('vector', 'age');")
    extensions = cursor.fetchall()
    print("インストール済み拡張機能:")
    for ext in extensions:
        print(f"  ✅ {ext[0]} (バージョン: {ext[1]})")

    # 2. テーブルの確認
    cursor.execute("""
    SELECT table_name
    FROM information_schema.tables
    WHERE table_schema = 'public'
    ORDER BY table_name;
    """)
    tables = cursor.fetchall()
    print(f"\n作成済みテーブル: {[table[0] for table in tables]}")

    # 3. グラフの確認
    cursor.execute("SET search_path = ag_catalog, \"$user\", public;")
    cursor.execute("SELECT name FROM ag_graph;")
    graphs = cursor.fetchall()
    print(f"作成済みグラフ: {[graph[0] for graph in graphs]}")

    # 4. データ件数の確認
    cursor.execute("SELECT COUNT(*) FROM documents;")
    doc_count = cursor.fetchone()[0]
    print(f"ドキュメント件数: {doc_count}")

    cursor.close()
    conn.close()

    # 5. バックアップの作成
    # backup_dir = '/content/drive/My Drive/minirag_backups'
    backup_dir = '/content/minirag_backups'
    !mkdir -p {backup_dir}

    timestamp = pd.Timestamp.now().strftime('%Y%m%d_%H%M%S')
    backup_file = f"{backup_dir}/minirag_complete_backup_{timestamp}.sql"

    # パスワード: postgres
    !pg_dump -h localhost -U postgres -d minirag_db > {backup_file}
    print(f"\n✅ バックアップ完了: {backup_file}")

    print("\n🎉 PostgreSQL + pgvector + Apache AGE の完全セットアップ成功！")

final_verification_and_backup()

=== 最終確認 ===
インストール済み拡張機能:
  ✅ vector (バージョン: 0.8.0)
  ✅ age (バージョン: 1.5.0)

作成済みテーブル: ['documents', 'entities', 'relationships']
作成済みグラフ: ['my_minirag_graph']
ドキュメント件数: 1
Password: 

✅ バックアップ完了: /content/minirag_backups/minirag_complete_backup_20250714_011652.sql

🎉 PostgreSQL + pgvector + Apache AGE の完全セットアップ成功！
